In [ ]:
import os
from datetime import date
from datetime import datetime

import pandas as pd
import duckdb


In [ ]:
from finlab import data

In [ ]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings
from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 欄數全展開
pd.set_option("display.max_columns", None)

## 外部資料讀取

In [ ]:
# 讀取台股期貨三大法人交易資訊
oi = data.get('futures_institutional_investors_trading_summary:空方未平倉口數', save_to_storage=True)
net_oi = data.get('futures_institutional_investors_trading_summary:多空未平倉口數淨額', save_to_storage=True)

In [ ]:
oi

In [ ]:
# 達人秀忠哥常用四項整體指標 ～ 外資未平倉口數
tw_net_oi_df = net_oi[[
    "臺股期貨_外資及陸資", "臺股期貨_投信", "臺股期貨_自營商",  # 大台指
    "小型臺指期貨_外資及陸資", "小型臺指期貨_投信", "小型臺指期貨_自營商",  # 小台指
    "金融期貨_外資及陸資", "金融期貨_投信", "金融期貨_自營商",  # 金融期貨
    "電子期貨_外資及陸資", "電子期貨_投信", "電子期貨_自營商",  # 電子期貨
]].fillna(0).reset_index()

In [ ]:
# tw_net_oi_df欄位名稱重新命名使用rename，臺股期貨_外資及陸資 -> 臺股期貨_外資
tw_net_oi_df.rename(columns={
    "date": "Date",
    "臺股期貨_外資及陸資": "臺股期貨_外資",
    "小型臺指期貨_外資及陸資": "小型臺指期貨_外資",
    "金融期貨_外資及陸資": "金融期貨_外資",
    "電子期貨_外資及陸資": "電子期貨_外資",
}, inplace=True)

tw_net_oi_df = tw_net_oi_df.tail(60)

## 資料留存ＤＢ

In [ ]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"
twstock_db_path

In [ ]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
table_name = "tw_futures_institutional_investors_trading_summary"

In [ ]:
# # 檔案整批寫入資料庫
# tw_net_oi_df.to_sql(table_name, conn_duckdb, if_exists="append", index=False)

In [ ]:
# 針對tw_net_oi_df，以Date單筆先確認表中，欄位Date沒有重複資料時，才進行單筆insert
# 篩選出最後60筆資料做更新即可
for i in range(len(tw_net_oi_df)):
    try:
        tw_net_oi_df.iloc[i:i+1].to_sql(table_name, conn_duckdb, if_exists="append", index=False)
    except Exception as e:
        # print(e)
        pass

In [ ]:
# 確認寫入結果
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc").fetch_df()

In [ ]:
# 關閉資料庫連線
conn_duckdb.close()

In [ ]:
# # 查詢dataframe columns 開頭為 WTE 的資料
# # TE代表電子期貨
# future_price_close.filter(regex='^TE', axis=1)

In [ ]:
# # TF代表金融期貨
# future_price_close.filter(regex='^TF', axis=1)

In [ ]:
# # 未平倉契約數
# # future_price_oi.filter(regex='^TF', axis=1)
# future_price_oi

In [ ]:
# future_txn_df = pd.read_csv("Daily_2024_08_16.csv", dtype=str, encoding="big5")
# future_txn_df

In [ ]:
# future_txn_df.query('商品代號 == "BRF"')

In [ ]:
# future_txn_df["商品代號"] = future_txn_df["商品代號"].str.strip()
# future_txn_df["到期月份(週別)"] = future_txn_df["到期月份(週別)"].str.strip()
# # future_txn_df["商品代號"].unique()

In [ ]:
# te_df = future_txn_df[
#     (future_txn_df["商品代號"] == "TE") & (future_txn_df["到期月份(週別)"].str.contains("202408"))
# ]

In [ ]:
# future_txn_df["到期月份(週別)"].unique()

In [ ]:
# k_df = te_df[
#     (te_df["成交時間"] >= "124500") & (te_df["成交時間"] <= "134500") & (te_df["成交價格"] > "0")
# ].sort_values("成交時間")
# k_df

In [ ]:
# k_df.成交價格.min()